In [1]:
import pandas as pd
import os
import zipfile
import json
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
from faiss import IndexFlatL2
from sklearn.metrics.pairwise import cosine_similarity


ModuleNotFoundError: No module named 'torch'

In [7]:
tf.debugging.set_log_device_placement(True)
df = pd.read_csv('archive2/nq_small.csv')

In [8]:
df.head(10)

,answer,context,id,question,answer_start,p0,p1,alen
0,103 Hz,\n\nThe hertz (symbol: Hz) is the derived unit...,532428134682627320,1 kilohertz is equal to how many hertz?,324,-0.469617,0.347219,6
1,1/18th the size,1:18 scale diecast replicas are 1/18th the siz...,-4516564998123606940,How big is a 1 18 scale model?,32,-1.959410,1.614884,15
2,380 AD,Nicene Christianity became the state church of...,4884836952562841253,When did christianity become official religion...,98,0.222497,-0.266590,6
3,seventeenth-century,Salt and pepper is the common name for edible ...,6442551588281709780,When did salt and pepper became a pair?,352,-0.315290,0.215326,19
4,Harry S. Truman,"\n\nHarry S. Truman[b] (May 8, 1884 – December...",9214285402926650474,Who was president when world war 2 ended?,2,-0.080704,-0.262564,15
5,Call of Duty,Call of Duty is a first-person shooter video g...,-59817003176266817,What was the first call of duty game ever made?,0,0.095545,-0.257527,12
6,48 BC,The Library of Alexandria was one of the large...,2093732342481935035,When did the library of alexandria burn down?,1015,-0.536527,0.298526,5
7,employees injured in the course of employment,Workers' compensation is a form of insurance p...,528868946182004081,When does an employee qualify for workers comp...,96,-0.557063,0.329259,45
8,universal motors,The universal motor is so named because it is ...,7651063123716449392,What type of motor used in vacuum cleaner?,1330,-0.770690,0.507451,16
9,the Indian painter Abanindranath Tagore,\n\nBharat Mata is a work painted by the India...,-5586968952968112296,Who was the painter of this famous painting bh...,35,0.045844,-0.234746,39


## Trial Retriever

In [12]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from faiss import IndexFlatL2
from sklearn.metrics.pairwise import cosine_similarity




tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
p_model = BertModel.from_pretrained('p_mod',from_tf=True)
q_model = BertModel.from_pretrained('q_mod',from_tf=True)

# Encode passages and questions
passage_encodings = []
question_encodings = []

for index, row in df.iterrows():
    passage_tokens = tokenizer.encode(row['context'], add_special_tokens=True, max_length=64, truncation=True, padding='max_length')
    question_tokens = tokenizer.encode(row['question'], add_special_tokens=True, max_length=64, truncation=True, padding='max_length')
    
    with torch.no_grad():
        passage_output = p_model(torch.tensor([passage_tokens]))[0]
        question_output = q_model(torch.tensor([question_tokens]))[0]
    
    passage_encodings.append(passage_output)
    question_encodings.append(question_output)


passage_encodings = torch.stack(passage_encodings).numpy()
question_encodings = torch.stack(question_encodings).numpy()


d = passage_encodings.shape[-1]  
index = IndexFlatL2(d)
index.add(passage_encodings)


accuracy_matrix = []

for index, row in df.iterrows():
    question_tokens = tokenizer.encode(row['question'], add_special_tokens=True, max_length=64, truncation=True, padding='max_length')
    
    with torch.no_grad():
        question_output = q_model(torch.tensor([question_tokens]))[0]

    question_encoding = question_output.numpy()
    question_encoding = question_encoding.reshape(1, -1)  
    
    D, I = index.search(question_encoding, k=1)  
    
    top_passage_context = df.iloc[I[0][0]]['context']
    

    sim_score = cosine_similarity(question_encoding, passage_encodings[I[0][0]].reshape(1, -1))
    
    accuracy_matrix.append((row['question'], top_passage_context, sim_score))



Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZerosLike in device /job:localhos

All TF 2.0 model weights were used when initializing BertModel.

All the weights of BertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZerosLike in device /job:localhos

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BiasAdd in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _MklMatMul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BiasAdd in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/re

Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _MklMatMul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BiasAdd in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _MklMatMul in device /job:localhost/repli

...

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /j

Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:loc

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /j

All TF 2.0 model weights were used when initializing BertModel.

All the weights of BertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


ValueError: too many values to unpack (expected 2)

In [29]:
print("Shape of passage_encodings:", passage_encodings.shape)


Shape of passage_encodings: (10658, 1, 64, 768)


In [13]:
passage_encodings = passage_encodings.reshape(passage_encodings.shape[0], -1)

d = passage_encodings.shape[-1]  # Get the vector dimension from the output
index = IndexFlatL2(d)
index.add(passage_encodings)

# Compute accuracy matrices
accuracy_matrix = []
ind=0
for index, row in df.iterrows():
    question_tokens = tokenizer.encode(row['question'], add_special_tokens=True, max_length=64, truncation=True, padding='max_length')
    
    with torch.no_grad():
        question_output = q_model(torch.tensor([question_tokens]))[0]

    question_encoding = question_output.numpy()
    question_encoding = question_encoding.reshape(1, -1)  # Reshape to match Faiss input
    
    D, I = index.search(question_encoding, k=1)  # Retrieve top k passages
    if ind<10:
        print(D,I)
        ind+=1
    # Retrieve the context of the top passage
    top_passage_context = df.iloc[I[0][0]]['context']
    
    # Compute similarity between the retrieved passage and the ground truth passage
    sim_score = cosine_similarity(question_encoding, passage_encodings[I[0][0]].reshape(1, -1))
    
    accuracy_matrix.append((row['question'], top_passage_context, sim_score))

AttributeError: 'int' object has no attribute 'search'

In [14]:
# Create Faiss index
d = passage_encodings.shape[-1]  # Get the vector dimension from the output
index = IndexFlatL2(d)
index.add(passage_encodings)


accuracy_matrix = []

# Debugging step: Print the index type before the loop to ensure it's not reassigned
print("Index type before loop:", type(index))

for _, row in df.iterrows():
    question_tokens = tokenizer.encode(row['question'], add_special_tokens=True, max_length=64, truncation=True, padding='max_length')
    
    with torch.no_grad():
        question_output = q_model(torch.tensor([question_tokens]))[0]

    question_encoding = question_output.numpy()
    question_encoding = question_encoding.reshape(1, -1)  
    
   
    print("Index type inside loop:", type(index))
    
    

    D, I = index.search(question_encoding, k=1)  # Retrieve top k passages
    
    # Retrieve the context of the top passage
    top_passage_context = df.iloc[I[0][0]]['context']
    
    # Compute similarity between the retrieved passage and the ground truth passage
    sim_score = cosine_similarity(question_encoding, passage_encodings[I[0][0]].reshape(1, -1))
    
    accuracy_matrix.append((row['question'], top_passage_context, sim_score))



Index type before loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlat

Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlat

Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlat

Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlat

Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlat

...

Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlat

Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlat

Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlat

Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlat

Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Index type inside loop: <class 'faiss.swigfaiss_avx2.IndexFlat

In [15]:
# Sort the accuracy matrix based on similarity score in descending order
accuracy_matrix_sorted = sorted(accuracy_matrix, key=lambda x: x[2], reverse=True)

# Print the top 10 entries
print("Top 10 entries:")
for i in range(10):
    print(f"Question: {accuracy_matrix_sorted[i][0]}")
    print(f"Top Passage Context: {accuracy_matrix_sorted[i][1]}")
    print(f"Similarity Score: {accuracy_matrix_sorted[i][2][0][0]}")
    print()


Top 10 entries:
Question: Who won the match sri lanka or india?
Top Passage Context: 

The Sydney funnel-web spider (Atrax robustus) is a species of venomous mygalomorph  spider native to eastern Australia, usually found within a 100 km (62 mi) radius of Sydney. It is a member of a group of spiders known as Australian funnel-web spiders.  Its bite is capable of causing serious injury or death in humans if left untreated.[3]

The Sydney funnel-web is medium to large in size, with body length ranging from 1 to 5 cm (0.4 to 2 in). Both sexes are glossy and darkly colored, ranging from blue-black, to black, to brown or dark-plum colored.

The Reverend Octavius Pickard-Cambridge was the first to describe the Sydney funnel-web spider, from a female specimen housed in the British Museum in 1877. Establishing the genus Atrax, he named it Atrax robustus.[4] The species name is derived from the Latin robustus "strong/sturdy/mature".[5] Some years later, William Joseph Rainbow described a male Sy

In [35]:
# Print all similarities
print("All Similarities:")
for entry in accuracy_matrix:
    print(f"Question: {entry[0]}")
    print(f"Top Passage Context: {entry[1]}")
    print(f"Similarity Score: {entry[2][0][0]}")
    print()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
# Sort the accuracy matrix based on similarity score
accuracy_matrix_sorted = sorted(accuracy_matrix, key=lambda x: x[2][0][0])

# Print the entry with the lowest similarity score
print("Entry with the lowest similarity score:")
print(f"Question: {accuracy_matrix_sorted[0][0]}")
print(f"Top Passage Context: {accuracy_matrix_sorted[0][1]}")
print(f"Similarity Score: {accuracy_matrix_sorted[0][2][0][0]}")
print()

# Print the entry with the highest similarity score
print("Entry with the highest similarity score:")
print(f"Question: {accuracy_matrix_sorted[-1][0]}")
print(f"Top Passage Context: {accuracy_matrix_sorted[-1][1]}")
print(f"Similarity Score: {accuracy_matrix_sorted[-1][2][0][0]}")


Entry with the lowest similarity score:
Question: What kind of economic system did west germany have?
Top Passage Context: 

The hertz (symbol: Hz) is the derived unit of frequency in the International System of Units (SI) and is defined as one cycle per second.[1] It is named for Heinrich Rudolf Hertz, the first person to provide conclusive proof of the existence of electromagnetic waves. Hertz are commonly expressed in multiples: kilohertz (103 Hz, kHz), megahertz (106 Hz, MHz), gigahertz (109 Hz, GHz), terahertz (1012 Hz, THz), petahertz (1015 Hz, PHz), and exahertz (1018 Hz, EHz).

Some of the unit's most common uses are in the description of sine waves and musical tones, particularly those used in radio- and audio-related applications. It is also used to describe the speeds at which computers and other electronics are driven.

The hertz is defined as one cycle per second. The International Committee for Weights and Measures defined the second as "the duration of 9 192 631 770 peri

In [37]:
# Calculate the average similarity score
total_similarity = sum(entry[2][0][0] for entry in accuracy_matrix)
average_similarity = total_similarity / len(accuracy_matrix)

print(f"Average Similarity Score: {average_similarity}")


Average Similarity Score: 0.9867967928265693


In [38]:

num_null_rows = df.isnull().any(axis=1).sum()

print("Number of null rows in the DataFrame:", num_null_rows)


Number of null rows in the DataFrame: 0


## Reader

In [26]:
from transformers import BertForQuestionAnswering, BertTokenizer, AdamW
import torch

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

# After fine-tuning, you can use the model for inference
def extract_answer(question, passage):
    inputs = tokenizer(question, passage, return_tensors="pt", truncation=True)
    outputs = model(**inputs)

    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Get the most probable start and end positions
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits) + 1
    print(start_index,end_index)
    answer_tokens = inputs["input_ids"][0][min(start_index,end_index):max(start_index,end_index)]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    return answer

# Example usage
question = "What is the capital of France?"
passage = "The capital of France is Paris."
answer = extract_answer(question, passage)
print("Answer:", answer)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor(1) tensor(7)
Answer: what is the capital of france


In [27]:
text = r"""Japan is the eleventh-most populous country in the world, as well as one of the most densely populated and urbanized.
 About three-fourths of the country's terrain is mountainous, concentrating its population of 125.57 million on narrow coastal plains. 
 Japan is divided into 47 administrative prefectures and eight traditional regions.
 Osaka has a big population of 16 million. 
 The Greater Tokyo Area is the most populous metropolitan area in the world, with more than 37.4 million residents. Japan has made Anime which is the best thing to have happened.The best anime is Rezero
"""

In [24]:
text=r"""Caffeine, the stimulant in coffee, has been called “the most widely used psychoactive substance on Earth.” Snyder, Daly and Bruns have recently proposed that caffeine affects behavior by countering the activity in the human brain of a naturally occurring chemical called adenosine. Adenosine normally depresses neuron firing in many areas of the brain. It apparently does this by inhibiting the release of neurotransmitters, chemicals that carry nerve impulses from one neuron to the next. Like many other agents that affect neuron firing, adenosine must first bind to specific receptors on neuronal membranes. There are at least two classes of these receptors, which have been designated A1 and A2. Snyder et al propose that caffeine, which is structurally similar to adenosine, is able to bind to both types of receptors, which prevents adenosine from attaching there and allows the neurons to fire more readily than they otherwise would. For many years, caffeine’s effects have been attributed to its inhibition of the production of phosphodiesterase, an enzyme that breaks down the chemical called cyclic AMP. A number of neurotransmitters exert their effects by first increasing cyclic AMP concentrations in target neurons. Therefore, prolonged periods at the elevated concentrations, as might be brought about by a phosphodiesterase inhibitor, could lead to a greater amount of neuron firing and, consequently, to behavioral stimulation. But Snyder et al point out that the caffeine concentrations needed to inhibit the production of phosphodiesterase in the brain are much higher than those that produce stimulation. Moreover, other compounds that block phosphodiesterase’s activity are not stimulants. To buttress their case that caffeine acts instead by preventing adenosine binding, Snyder et al compared the stimulatory effects of a series of caffeine derivatives with their ability to dislodge adenosine from its receptors in the brains of mice. “In general,” they reported"""

In [28]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
def get_top_answers(possible_starts,possible_ends,input_ids):
  answers = []
  for start,end in zip(possible_starts,possible_ends):
    #+1 for end
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[start:end+1]))
    answers.append( answer )
  return answers  

def answer_question(question,context,topN):

    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    model_out = model(**inputs)
     
    answer_start_scores = model_out["start_logits"]
    answer_end_scores = model_out["end_logits"]

    possible_starts = np.argsort(answer_start_scores.cpu().detach().numpy()).flatten()[::-1][:topN]
    possible_ends = np.argsort(answer_end_scores.cpu().detach().numpy()).flatten()[::-1][:topN]
    
    #get best answer
    answer_start = torch.argmax(answer_start_scores)  
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    answers = get_top_answers(possible_starts,possible_ends,input_ids )

    return { "answer":answer,"answer_start":answer_start,"answer_end":answer_end,"input_ids":input_ids,
            "answer_start_scores":answer_start_scores,"answer_end_scores":answer_end_scores,"inputs":inputs,"answers":answers,
            "possible_starts":possible_starts,"possible_ends":possible_ends}

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
questions = [    
    "Where is the most populous metropolitan area in the world ?",
    "Where is the most populous city ?",
    "What is the most populous city ?",
    "Where is most populous in the world ?",
    "Where is most populous?",
    "What is Population of Tokyo ?",
    "Which city is most crowded in the world ?",
    "Which city has biggest population ?",
    "How is the terrain in Japan?",
    "What is the best anime?",
    "Which city has most population ?",
    ]

questions3 = [
    "Most populous city ?"  ,    
    #"Most populous city?"  ,       
    "Most populous city"  
    
    ]

for q in questions:
  answer_map = answer_question(q,text,5)    
  print("Question:",q)
  print("Answers:")
  [print((index+1)," ) ",ans) for index,ans in  enumerate(answer_map["answers"]) if len(ans) > 0 ]

Question: Where is the most populous metropolitan area in the world ?
Answers:
1  )  greater tokyo area
2  )  the greater tokyo
3  )  tokyo area is the most populous metropolitan area in the world
4  )  japan is the eleventh - most populous country in the world, as well as one of the most densely populated and urbanized. about three - fourths of the country ' s terrain is mountainous, concentrating its population of 125. 57 million on narrow coastal plains. japan is divided into 47 administrative prefectures and eight traditional regions. osaka has a big population of 16 million. the greater tokyo area is the most populous metropolitan area in the world,
5  )  japan is divided into 47 administrative prefectures and eight traditional regions. osaka has a big population of 16 million. the greater tokyo area is the most populous metropolitan area in the world, with more than 37. 4 million residents
Question: Where is the most populous city ?
Answers:
1  )  osaka
2  )  greater tokyo
4  )  